<a href="https://colab.research.google.com/github/Hotchapu13/MRI_Uganda_lab_notebooks/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **T2 Spin echo including Gradients**
Welcome to lab three where you shall explore some algorithms used to write MRI pulse sequences in Python.

This example illustrates the 2D multi-slice, Spin Echo (SE) https://mriquestions.com/se-vs-multi-se-vs-fse.html acquisition using the `pypulseq` library. This sequence is typically used for T<sub>2</sub> weighted imaging. A 2D Fourier transform can be used to reconstruct images from this acquisition. Read more about SE [here](http://mriquestions.com/se-vs-multi-se-vs-fse.html).

This notebook was expanded from the original: [You check it out here](https://github.com/imr-framework/pypulseq/blob/master/examples/notebooks/write_t2_se.ipynb)
---


**Objective:**  

To try to generate such a PSD (Pulse Sequence Diagram)

<p align="center">
  <img src="https://radiologykey.com/wp-content/uploads/2016/01/c00016_f016-003-9780323073547.jpg" alt="Spin Echo Pulse sequence diagram">
</p>

<p align="center" style="color: teal;">
  <i>Picture gotten from the page: <a href="https://radiologykey.com/spin-echo-imaging/" target="_blank" style="color: teal;">https://radiologykey.com/spin-echo-imaging/</a></i>
</p>


---
**Gradients reading material:** http://xrayphysics.com/spatial.html#gradients

## **INSTALL** `pypulseq`

PyPulseq is a vendor-neutral pulse sequence design tool for MRI, written in Python. It empowers researchers, students, and engineers to design sequences without being locked into proprietary vendor environments.

We won't dive deep into how it works but I just want you to get a feel of what goes on the algorithms we use for writing pulse sequences.

In [ ]:
!pip install git+https://github.com/imr-framework/pypulseq.git

## **IMPORT PACKAGES**

In [ ]:
from math import pi

import numpy as np

from pypulseq.calc_duration import calc_duration
from pypulseq.make_adc import make_adc
from pypulseq.make_delay import make_delay
from pypulseq.make_sinc_pulse import make_sinc_pulse
from pypulseq.make_trapezoid import make_trapezoid
from pypulseq.opts import Opts
from pypulseq.Sequence.sequence import Sequence

## **USER INPUTS**

These parameters are typically on the user interface of the scanner computer console

In [ ]:
n_avg = 1  # Number of averages
n_slices = 1  # Number of slices
n_x = 128
n_y = 128
fov = 220e-3  # mm
slice_thickness = 5e-3  # s
slice_gap = 15e-3  # s
rf_flip = 90  # degrees
rf_offset = 0
print('User inputs setup')

## **SYSTEM LIMITS**
Set the hardware limits and initialize sequence object

In [ ]:
system = Opts(
    max_grad=32,
    grad_unit='mT/m',
    max_slew=130,
    slew_unit='T/m/s',
    grad_raster_time=10e-6,
    rf_ringdown_time=10e-6,
    rf_dead_time=100e-6,
)
seq = Sequence(system)

## **TIME CONSTANTS**

In [ ]:
te = 100e-3  # s echo time
tr = 3  # s repetition time
tau = te / 2  # s time to 180 degree pulse
readout_time = 6.4e-3 #
pre_time = 8e-4  # s

### **RF Section**
In this section we shall look at both the RF and gradients. I have included more complex concepts but good thing you have enough time with this notebook to figure out and master what is going on.

<p align="center">
  <img src="https://64.media.tumblr.com/19c82c642910b0360dc06a0428d93407/tumblr_ppcak68FWd1tctq75o1_540.gif" alt="Avengers big three">
</p>

In [ ]:
flip90 = round(rf_flip * pi / 180, 3)
flip180 = 180 * pi / 180

# Creating the 90-degree pulse
rf90, gz90, _ = make_sinc_pulse(
    flip_angle=flip90,
    system=system,
    duration=4e-3,
    slice_thickness=slice_thickness,
    apodization=0.5,
    time_bw_product=4,
    return_gz=True,
    delay=system.rf_dead_time,
    use='excitation',
)

# Creating the 180-degree pulse for the echo
rf180, gz180, _ = make_sinc_pulse(
    flip_angle=flip180,
    system=system,
    duration=2.5e-3,
    slice_thickness=slice_thickness,
    apodization=0.5,
    time_bw_product=4,
    phase_offset=90 * pi / 180,
    return_gz=True,
    delay=system.rf_dead_time,
    use='refocusing',
)

### **READOUT**
Readout gradients and related events

In [ ]:
delta_k = 1 / fov
k_width = n_x * delta_k
gx = make_trapezoid(channel='x', system=system, flat_area=k_width, flat_time=readout_time)
adc = make_adc(num_samples=n_x, duration=gx.flat_time, delay=gx.rise_time, system=system)

### **PREPHASE AND REPHASE**

In [ ]:
phase_areas = (np.arange(n_y) - (n_y / 2)) * delta_k
gz_reph = make_trapezoid(channel='z', system=system, area=-gz90.area / 2, duration=2.5e-3)
gx_pre = make_trapezoid(channel='x', system=system, flat_area=k_width / 2, flat_time=readout_time / 2)
gy_pre = make_trapezoid(channel='y', system=system, area=phase_areas[-1], duration=2e-3)

### **SPOILER**

In [ ]:
gz_spoil = make_trapezoid(channel='z', system=system, area=gz90.area * 4, duration=pre_time * 4)

### **DELAYS**
Echo time (TE) and repetition time (TR). Here, TE is broken down into `delay1` and `delay2`.

In [ ]:
delay1 = tau - calc_duration(rf90) / 2 - calc_duration(gx_pre)
delay1 -= calc_duration(gz_spoil) - calc_duration(rf180) / 2
delay1 = make_delay(delay1)
delay2 = tau - calc_duration(rf180) / 2 - calc_duration(gz_spoil)
delay2 -= calc_duration(gx) / 2
delay2 = make_delay(delay2)
delay_tr = tr - calc_duration(rf90) / 2 - calc_duration(gx) / 2 - te
delay_tr -= calc_duration(gy_pre)
delay_tr = make_delay(delay_tr)
print(f'delay_1: {delay1.delay * 1000} ms')
print(f'delay_2: {delay1.delay * 1000} ms')
print(f'delay_tr: {delay_tr.delay} s')

### **CONSTRUCT SEQUENCE**
Construct sequence for one phase encode and multiple slices

In [ ]:
# Prepare RF offsets. This is required for multi-slice acquisition
delta_z = n_slices * slice_gap
z = np.linspace((-delta_z / 2), (delta_z / 2), n_slices) + rf_offset

for _k in range(n_avg):  # Averages
    for j in range(n_slices):  # Slices
        # Apply RF offsets
        freq_offset = gz90.amplitude * z[j]
        rf90.freq_offset = freq_offset

        freq_offset = gz180.amplitude * z[j]
        rf180.freq_offset = freq_offset

        for i in range(n_y):  # Phase encodes
            seq.add_block(rf90, gz90)
            gy_pre = make_trapezoid(channel='y', system=system, area=phase_areas[-i - 1], duration=2e-3)
            seq.add_block(gx_pre, gy_pre, gz_reph)
            seq.add_block(delay1)
            seq.add_block(gz_spoil)
            seq.add_block(rf180, gz180)
            seq.add_block(gz_spoil)
            seq.add_block(delay2)
            seq.add_block(gx, adc)
            gy_pre = make_trapezoid(channel='y', system=system, area=-phase_areas[-j - 1], duration=2e-3)
            seq.add_block(gy_pre, gz_spoil)
            seq.add_block(delay_tr)

In [ ]:
_k = 0  # Single average
j = 0  # Single slice (first one)
i = 0  # Single phase encode step (first one)

# Apply RF offsets for the single selected slice
freq_offset_90 = gz90.amplitude * z[j]
rf90.freq_offset = freq_offset_90

freq_offset_180 = gz180.amplitude * z[j]
rf180.freq_offset = freq_offset_180

# Construct the sequence for a single spin echo
seq = Sequence(system) # Re-initialize sequence object for clarity

# 90-degree pulse and slice select gradient
seq.add_block(rf90, gz90)

# Pre-phase gradients (adjusting gy_pre for the first phase encode step)
gy_pre_single = make_trapezoid(channel='y', system=system, area=phase_areas[-i - 1], duration=2e-3)
seq.add_block(gx_pre, gy_pre_single, gz_reph)

# Delay before 180 pulse
seq.add_block(delay1)

# Spoiler gradient
seq.add_block(gz_spoil)

# 180-degree pulse and slice select gradient
seq.add_block(rf180, gz180)

# Spoiler gradient after 180 pulse
seq.add_block(gz_spoil)

# Delay before readout
seq.add_block(delay2)

# Readout gradient and ADC
seq.add_block(gx, adc)

# Re-phase gradient after readout (adjusting gy_pre for the first phase encode step)
gy_post_single = make_trapezoid(channel='y', system=system, area=-phase_areas[-i - 1], duration=2e-3)
seq.add_block(gy_post_single, gz_spoil) # Adding spoiler after rephase

# TR delay
seq.add_block(delay_tr)

print('Sequence construction for a single spin echo completed.')

In [ ]:
print(f'Current TE: {te} s')
print(f'Current TR: {tr} s')

# Calculate expected timings based on current delays and pulse durations
te_calc = calc_duration(rf90) + calc_duration(gx_pre) + delay1.delay + calc_duration(gz_spoil) + calc_duration(rf180) + calc_duration(gz_spoil) + delay2.delay + calc_duration(gx)/2

print(f'Calculated TE based on block durations: {te_calc} s')

tr_calc = te + calc_duration(gx)/2 + calc_duration(gy_post_single) + calc_duration(gz_spoil) + delay_tr.delay
print(f'Calculated TR based on block durations: {tr_calc} s')

**The calculated TE is slightly off from the desired TE. Adjust TE and TR if necessary and re-calculate the delays to ensure the key events align with TE and TR.**



In [ ]:
"""
Adjust TE and TR if needed. The current calculated values are close to the desired ones,but we will recalculate delays to ensure accuracy.
No adjustment needed for TE and TR as the calculated values are very close to the target values.

Re-calculate delays based on the desired TE and TR and the duration of the sequence blocks.
TE = duration(rf90)/2 + duration(gx_pre) + delay1 + duration(gz_spoil) + duration(rf180)/2 + duration(rf180)/2 + duration(gz_spoil) + delay2 + duration(gx)/2
TE/2 = duration(rf90)/2 + duration(gx_pre) + delay1 + duration(gz_spoil) + duration(rf180)/2
"""

delay1 = te/2 - calc_duration(rf90)/2 - calc_duration(gx_pre) - calc_duration(gz_spoil) - calc_duration(rf180)/2
delay1 = make_delay(delay1)

# TE = TE/2 + duration(rf180)/2 + duration(gz_spoil) + delay2 + duration(gx)/2
delay2 = te/2 - calc_duration(rf180)/2 - calc_duration(gz_spoil) - calc_duration(gx)/2
delay2 = make_delay(delay2)

"""
TR = duration(rf90)/2 + duration(gx_pre) + delay1 + duration(gz_spoil) + duration(rf180) + duration(gz_spoil) + delay2 + duration(gx)/2 + duration(gy_post_single) + duration(gz_spoil) + delay_tr + duration(rf90)/2 (for the next TR)
TR = TE + duration(gx)/2 + duration(gy_post_single) + duration(gz_spoil) + delay_tr
"""
delay_tr = tr - te - calc_duration(gx)/2 - calc_duration(gy_post_single) - calc_duration(gz_spoil)
delay_tr = make_delay(delay_tr)


print(f'delay_1: {delay1.delay * 1000} ms')
print(f'delay_2: {delay2.delay * 1000} ms')
print(f'delay_tr: {delay_tr.delay} s')

# Reconstruct the sequence with potentially adjusted delays
seq = Sequence(system) # Re-initialize sequence object

# 90-degree pulse and slice select gradient
seq.add_block(rf90, gz90)

"""
Pre-phase gradients (adjusting gy_pre for the first phase encode step)
gy_pre_single was already defined in the previous step, no need to redefine if i=0 is fixed.
"""
seq.add_block(gx_pre, gy_pre_single, gz_reph)

# Delay before 180 pulse
seq.add_block(delay1)

# Spoiler gradient
seq.add_block(gz_spoil)

# 180-degree pulse and slice select gradient
seq.add_block(rf180, gz180)

# Spoiler gradient after 180 pulse
seq.add_block(gz_spoil)

# Delay before readout
seq.add_block(delay2)

# Readout gradient and ADC
seq.add_block(gx, adc)

seq.add_block(gy_post_single, gz_spoil) # Adding spoiler after rephase

# TR delay
seq.add_block(delay_tr)

print('Sequence reconstructed with potentially adjusted delays.')

### Visualize key events

Visualize the key events of the spin echo sequence, highlighting the timing of the 90-degree pulse, the 180-degree pulse, and the ADC window.


In [ ]:
seq.plot(time_range=(0, te + calc_duration(gx) + calc_duration(gy_post_single) + calc_duration(gz_spoil) + 0.01))

## Assignments

### Assignment 1
### Calculate Slice-Selection Gradient Amplitude

Based on the provided information, we can calculate the amplitude of the slice-selection gradient using the formula: **BW=γ⋅G⋅Δz**  

Hence:  
Gradient Amplitude (**G**) = Bandwidth(**BW**) / (Gamma(**γ**) * Slice Thickness(**Δz**))

Where:
- Bandwidth (BW) = 1000 Hz (1 kHz)
- Slice Thickness = 5 mm = 0.005 m
- Gamma (Gyromagnetic Ratio for Hydrogen) ≈ 42.57 MHz/T ≈ 42.57 * 10e6 Hz/T

The fundamental relationship between bandwidth, gradient, and slice thickness:   

---
**Expected Ans:** *4.70 mT/m*


In [ ]:
# Constants
gamma = 42.57e6  # Gyromagnetic ratio for Hydrogen in Hz/T
bandwidth = 1000  # Bandwidth in Hz
slice_thickness = 0.005  # Slice thickness in meters

# Complete the calculation
gradient_amplitude = bandwidth/(gamma * slice_thickness)

print(f"The Gradient Amplitude (G): {gradient_amplitude * 1000:.2f}")

### Assignment 2
### What is the minimum achievable slice thickness given a minimum RF BW = 426 Hz and a maximum gradient Gz = 10 mT/m?

Based on the provided information, we can calculate the minimum achievable slice thickness using the rearranged formula from before:

Slice Thickness(**Δz**) = Bandwidth(**BW**) / (Gamma(**γ**) * Gradient Amplitude (**G**))


Where:
- Bandwidth (BW) = 426 Hz
- Gradient Amplitude (Gz) = 10 mT/m = 0.01 T/m
- Gamma (Gyromagnetic Ratio for Hydrogen) ≈ 42.57 MHz/T ≈ 42.57 * 10e6 Hz/T

---

**Ans:** *1.00 mm*

### Assignment 3

Read: http://xrayphysics.com/sequences.html only upto Spin Echo, the concepts below are still beyond our current scope of coverage but we shall cover them in the nearby future.

## Conclusion

In this lab, we explored the fundamental concepts behind writing MRI pulse sequences using the `pypulseq` library. We focused on the 2D multi-slice Spin Echo (SE) sequence, a common technique for T2-weighted imaging. Through this exercise, we gained insight into how algorithms are used to design sequences and the role of different components like RF pulses, gradients, and timing parameters (TE and TR). We also practiced calculating key parameters such as slice-selection gradient amplitude and minimum achievable slice thickness based on system limitations. This hands-on experience provides a foundation for understanding more complex MRI pulse sequences and their implementation.

I know this was heavy but tough times create tough men so use this time to relax and get ready for signal processing.

<p align="center">
  <img src="https://www.masala.com/cloud/2023/08/04/images-17.jpeg" alt="Oppennheimer">
</p>

## 📖 References

*   Ravi, Keerthi, Sairam Geethanath, and John Vaughan. "PyPulseq: A Python Package for MRI Pulse Sequence Design." *Journal of Open Source Software* 4.42 (2019): 1725.
*   Ravi, Keerthi Sravan, et al. "Pulseq-Graphical Programming Interface: Open source visual environment for prototyping pulse sequences and integrated magnetic resonance imaging algorithm development." *Magnetic resonance imaging* 52 (2018): 9-15.
*   Layton, Kelvin J., et al. "Pulseq: a rapid and hardware‐independent pulse sequence prototyping framework." *Magnetic resonance in medicine* 77.4 (2017): 1544-1552.